In [ ]:
#importing modules
import numpy as np
import matplotlib.pyplot as plt; plt.ioff()

from astropy.io import fits as pyfits
from astropy import wcs
from astropy.table import Table, join, vstack
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from scipy.optimize import curve_fit

import emcee
import corner 

from os.path import exists
nan= float('nan')

pi= np.pi
e= np.e

plt.close('all')
c= 3.*10**8
h= 6.62*10**-34
k= 1.38*10**-23



PATH='/Users/jansen/My Drive/Astro/'


In [ ]:
import pyneb as pn

Flux_halpha = 4e-17
Flux_hbeta = 1e-17

Intrinsic_rat = 2.86

rc = pn.RedCorr(law= 'CCM89', R_V=2.505)
rc.setCorr((Flux_halpha/Flux_hbeta)/Intrinsic_rat, wave1=6563, wave2=4861)


Flux_halpha_cor  = Flux_halpha* rc.getCorr(6563)

print('Av = ', rc.AV)
print('Flux of Halpha corrected for extinction: ', Flux_halpha_cor, 'ergs/s/cm-2')



## SFR

In [ ]:
redshift = 3.4
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u 
lum = (Flux_halpha_cor*u.erg/u.s/u.cm**2) * 4*np.pi*(cosmo.luminosity_distance(redshift))**2  
lum = lum.to(u.erg/u.s)  # Convert to ergs/s
print('Luminosity', lum,) #'ergs/s')

# SFR calibration in https://ui.adsabs.harvard.edu/abs/2012ARA%26A..50..531K/abstract
# Table 1, page 17 of PDF, 547 of the review
CX = 1
SFR = lum.value * CX

print(  'SFR = ', SFR, 'Msun/yr')